In [1]:
# 模块导入
import pandas as pd

In [2]:
# 读入txt 每一行的数据形如:1,30.624806,104.136604,1,2014/8/3 21:18:46
# 1:出租车ID 2:维度 3:经度 4:载客情况(1为有,0为无) 5:时间
# 读入txt文件，将每个出租车的数据分别存入一个csv文件中
file_path="20140803_train.txt"
column_names=['taxi_id','latitude','longitude','passenger','time']
# 读入txt文件1
df=pd.read_csv(file_path,sep=',',header=None,names=column_names)


# 数据预处理
计算指标:里程利用率、时间利用率、空驶时间

In [3]:
# 提取出taxi_id为0\1\2\3的数据
df_selected = df[df['taxi_id']<=3].copy() # 测试用
# df_selected = df

In [4]:
# 把time列形如:2014/8/3 06:00:53的数据转换成易于处理的形式
df_selected['time']=pd.to_datetime(df_selected['time'],format='%Y/%m/%d %H:%M:%S')
# 增加一列time_s,表示以秒为单位的时间
df_selected['time_s']=df_selected['time'].dt.hour*3600+df_selected['time'].dt.minute*60+df_selected['time'].dt.second
# 以taxi_id为第一关键字，以time为第二关键字，对数据进行排序
df_selected=df_selected.sort_values(by=['taxi_id','time_s'])

In [5]:
# 每辆车的GPS数据采取的时间间隔最长和最短值
time_interval_min = df_selected.groupby('taxi_id')['time_s'].diff().min()
time_interval_max = df_selected.groupby('taxi_id')['time_s'].diff().max()
print('time_interval_min:',time_interval_min)
print('time_interval_max:',time_interval_max)

# 车辆总数
taxi_num = df_selected['taxi_id'].unique().shape[0]
print('taxi_num:',taxi_num)

# 找出每辆车 时间间隔为time_interval_min的数据
df_test = df_selected.groupby('taxi_id').apply(lambda x:x[x['time_s'].diff()==time_interval_min])
print(df_test)
# 找出每辆车 时间间隔为time_interval_max的数据
df_test = df_selected.groupby('taxi_id').apply(lambda x:x[x['time_s'].diff()==time_interval_max])
print(df_test)


time_interval_min: 1.0
time_interval_max: 301.0
taxi_num: 3
              taxi_id   latitude   longitude  passenger                time  \
taxi_id                                                                       
1       797         1  30.650550  104.126311          1 2014-08-03 09:20:23   
        1632        1  30.655733  104.115281          1 2014-08-03 09:27:05   
        1639        1  30.656019  104.115268          1 2014-08-03 09:27:08   
        1642        1  30.656112  104.115263          1 2014-08-03 09:27:09   
        1645        1  30.656211  104.115261          1 2014-08-03 09:27:10   
        1646        1  30.656309  104.115260          1 2014-08-03 09:27:11   
        1844        1  30.680516  104.100271          1 2014-08-03 09:53:54   
        1900        1  30.683680  104.100539          1 2014-08-03 10:10:20   
        1712        1  30.670089  104.106788          1 2014-08-03 10:15:03   
        795         1  30.650510  104.126379          1 2014-08-03 14:4

In [6]:
# 定义一个函数，用来计算两个经纬度之间的距离
from geopy.distance import geodesic

def get_distance(point1, point2):
    """
    计算两个经纬度之间的距离
    :param point1: 起点经纬度 (纬度, 经度)
    :param point2: 终点经纬度 (纬度, 经度)
    :return: 距离（km）
    """
    return geodesic(point1, point2).km

In [24]:
column_names=['taxi_id','passenger','start_time','end_time','start_latitude','start_longitude','end_latitude','end_longitude','distance']
# 当passenger列为1时，表示载客，为0时表示无客,以此来把轨迹GPS划分为载客轨迹和无客轨迹
# 并记录载客轨迹和无客轨迹的起始时间和终止时间(s)，起始地点和终止地点、轨迹长度(km)(不是位移)
trip_df=pd.DataFrame(columns=column_names)

# 合并df_selected中连续的载客数据或无客数据，形成一条轨迹
for cur_taxi,group in df_selected.groupby('taxi_id'):
    last_row = group.iloc[0].to_dict()
    cur_passenger = last_row['passenger']
    cur_distance = 0
    start_latitude = last_row['latitude']
    start_longitude = last_row['longitude']
    start_time = last_row['time_s']
    
    for i in range(1,group.shape[0]):# 从第二行开始遍历
        cur_row = group.iloc[i].to_dict()
        
        if cur_passenger != cur_row['passenger']:
            # 如果载客情况发生了变化则表示本段轨迹记录完成，记录其终止时间、终止地点
            cur_distance += get_distance((cur_row['latitude'], cur_row['longitude']), (last_row['latitude'], last_row['longitude']))
            end_time = last_row['time_s']
            end_latitude =  last_row['latitude']
            end_longitude =  last_row['longitude']
            new_row = pd.DataFrame({'taxi_id': cur_taxi, 'passenger': cur_passenger, 'start_time': start_time, 'end_time': end_time, \
                                    'start_longitude': start_longitude, 'start_latitude': start_latitude,\
                                    'end_longitude': end_longitude, 'end_latitude': end_latitude, 'distance': cur_distance},index=[0])
            trip_df = pd.concat([trip_df,new_row],ignore_index=True)
            # 重置轨迹的起始时间、起始地点、载客情况、轨迹长度
            # 轨迹的起始时间
            start_time = cur_row['time_s']
            # 轨迹的起始地点
            start_latitude = cur_row['latitude']
            start_longitude = cur_row['longitude']
            # 轨迹的载客情况
            cur_passenger = cur_row['passenger']
            # 轨迹的长度
            cur_distance = 0
        else: # 如果载客情况没有发生变化，则添加轨迹长度，继续记录轨迹
            cur_distance += get_distance((cur_row['latitude'], cur_row['longitude']), (last_row['latitude'], last_row['longitude']))
        last_row = cur_row

In [27]:
# 根据trip_df中的数据，计算每辆出租车的里程利用率、时间利用率、总空驶时间
taxi_df = pd.DataFrame(columns=['taxi_id', 'mileage_utilization', 'time_utilization', 'empty_time'])
for taxi_id in trip_df['taxi_id'].unique():
    # 计算每辆出租车的里程利用率、时间利用率、总空驶时间
    mileage_utilization = trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==1)]['distance'].sum() / \
        trip_df[trip_df['taxi_id']==taxi_id]['distance'].sum()
    time_utilization = (trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==1)]['end_time'].sum() - \
                        trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==1)]['start_time'].sum()) / \
                            (trip_df[trip_df['taxi_id']==taxi_id]['end_time'].sum() - \
                             trip_df[trip_df['taxi_id']==taxi_id]['start_time'].sum())
    empty_time = trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==0)]['end_time'].sum() - \
        trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==0)]['start_time'].sum()
    # 将计算结果记录到taxi_df中
    new_row = pd.DataFrame({'taxi_id': taxi_id, 'mileage_utilization': mileage_utilization, 'time_utilization': time_utilization, 'empty_time': empty_time}, index=[0])
    taxi_df = pd.concat([taxi_df, new_row], ignore_index=True)


计算目标:成都市一天内的打车需求时间分布

计算目标:成都市一天内的打车需求地区分布